In [2]:
using Pkg
Pkg.activate("../")
using Distributions, EasyStream
include("/home/coiote/Codes/jl/DMClassification/src/DMClassification.jl");

 Activating environment at `~/Codes/jl/EasyStream/Project.toml`


In [3]:
path = "../datasets/sinthetic/"
window_size = 2000
n_avaiable_labels = 150
name_files = split(read(`ls $path`, String), "\n")[1:end-1]
name_files = [name_files[i] for i in [1 3 4 5 8 11 13]]

1×7 Array{SubString{String},2}:
 "1CDT.csv"  "1CSurr.csv"  "2CDT.csv"  …  "FG_2C_2D.csv"  "MG_2C_2D.csv"

In [4]:
streams = Array{Any, 1}()
for name_file in name_files
    dataset = CSVFiles.load(path * name_file, header_exists=false) |> DataFrame
    coerce!(dataset, autotype(dataset))
    samples = Matrix{Float64}(dataset[:,1:end-1])
    labels = categorical(dataset[:,end])
    push!(streams, EasyStream.Stream(window_size, n_avaiable_labels, samples, labels, name=split(name_file, ".")[1]));
end

In [5]:
function teste(alpha)
    model = DMClassification.DMClassifier(forgetting_type=:DT2, alpha=alpha)
    teste = [0. for i in 1:length(streams)]
    Threads.@threads for i in 1:length(streams)
        result = EasyStream.evaluate(streams[i], model, measures = accuracy, output=[:table])
        println("Precisão em ", streams[i].name,": ",result.tables[1][1,end])
        teste[i] = result.tables[1][1,end]
    end
    
    return 1 - mean(teste)
end

teste (generic function with 1 method)

In [ ]:
function gd()
    iteracoes = 5
    alpha_atual = rand() * 5
    taxa_avanco = 0.8
    alpha_anterior = nothing
    for i in 1:iteracoes
        alpha_anterior = alpha_atual
        y = teste(alpha_atual)
        println("Alfa Atual: ", alpha_atual)
        println("Erro Médio: ", y)
        alpha_atual = alpha_atual - taxa_avanco * y
    end
end
gd()

In [ ]:
alpha = 0.31
model = DMClassification.DMClassifier(forgetting_type=:DT2, alpha=alpha)

In [6]:
function gd()
    iteracoes = 15
    alpha_atual = rand() * 5
    taxa_avanco = 0.8
    alpha_anterior = nothing
    for i in 1:iteracoes
        alpha_anterior = alpha_atual
        y = teste(alpha_atual)
        println("Alfa Atual: ", alpha_atual)
        println("Erro Médio: ", y)
        alpha_atual = alpha_atual - taxa_avanco * y
    end
end
gd()

Precisão em 1CDT: 0.9929968454258675
Precisão em 2CHT: 0.7230283911671924
Precisão em 2CDT: 0.6325552050473187
Precisão em 

┌ Warning: `names!(df::AbstractDataFrame, vals::Vector{Symbol}; makeunique::Bool = false)` is deprecated, use `rename!(df, vals, makeunique = makeunique)` instead.
│   caller = evaluate(::EasyStream.Stream, ::Main.DMClassification.DMClassifier; measures::MLJBase.Accuracy, output::Array{Symbol,1}, header::Nothing, steps::Int64) at evaluate.jl:98
└ @ EasyStream /home/coiote/Codes/jl/EasyStream/src/evaluate.jl:98


1CSurr: 0.48052890283496275
Precisão em 4CRE-V1: 0.26461353624349215
Precisão em MG_2C_2D: 0.6099274455841881
Precisão em FG_2C_2D: 0.711883912934701
Alfa Atual: 4.737042341960194
Erro Médio: 0.3692093943946111
Precisão em 2CDT: 0.6996845425867508
Precisão em 2CHT: 0.7230914826498422
Precisão em 1CDT: 0.9931230283911672
Precisão em 1CSurr: 0.8727078156458019
Precisão em 4CRE-V1: 0.26460552663195835
Precisão em FG_2C_2D: 0.7132899674756067
Precisão em MG_2C_2D: 0.5921841381035777
Alfa Atual: 4.441674826444506
Erro Médio: 0.30590192835932783
Precisão em 2CHT: 0.7305362776025237
Precisão em 2CDT: 0.7579810725552051
Precisão em 1CDT: 0.993186119873817
Precisão em 1CSurr: 0.4560607984328805
Precisão em 4CRE-V1: 0.26461353624349215
Precisão em FG_2C_2D: 0.7138203652739554
Precisão em MG_2C_2D: 0.628001000750563
Alfa Atual: 4.196953283757043
Erro Médio: 0.3508286898953662
Precisão em 2CDT: 0.7543217665615142
Precisão em 2CHT: 0.7589905362776025
Precisão em 1CDT: 0.9933753943217666
Precisão em